## Setup Connection + paths

In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

ROOT = Path.cwd()
while ROOT.name.lower() != "ecommerceproject" and not (ROOT / "analysis").exists():
    if ROOT.parent == ROOT:
        raise RuntimeError("Open notebook from inside C:\\dev\\ecommerceproject")
    ROOT = ROOT.parent

load_dotenv(ROOT / ".env")
db_url = os.getenv("DATABASE_URL")
engine = create_engine(db_url)

SQL_DIR = ROOT / "sql"
MARTS_DIR = SQL_DIR / "marts"

print("ROOT:", ROOT)
print("MARTS_DIR:", MARTS_DIR)  # C:\dev\ecommerceproject\sql\marts


ROOT: c:\dev\ecommerceproject
MARTS_DIR: c:\dev\ecommerceproject\sql\marts


### Utility to run a .sql file

In [3]:
def run_sql_file(path: Path):
    sql = path.read_text(encoding="utf-8")
    with engine.begin() as conn:
        conn.execute(text(sql))
    print("✅ Ran:", path)

def table_count(full_name: str) -> int:
    q = f"SELECT COUNT(*)::int AS n FROM {full_name};"
    with engine.connect() as conn:
        return conn.execute(text(q)).scalar()


## Run Marts

In [5]:
run_sql_file(MARTS_DIR / "mart_order_items_base.sql")
run_sql_file(MARTS_DIR / "mart_order_items_base_indexes.sql")

run_sql_file(MARTS_DIR / "mart_top_products.sql")
run_sql_file(MARTS_DIR / "mart_traffic_source_quality.sql")
run_sql_file(MARTS_DIR / "mart_fulfillment_deciles.sql")
run_sql_file(MARTS_DIR / "mart_monthly_trends.sql")
run_sql_file(MARTS_DIR / "mart_cohorts_new_vs_repeat.sql")


✅ Ran: c:\dev\ecommerceproject\sql\marts\mart_order_items_base.sql
✅ Ran: c:\dev\ecommerceproject\sql\marts\mart_order_items_base_indexes.sql
✅ Ran: c:\dev\ecommerceproject\sql\marts\mart_top_products.sql
✅ Ran: c:\dev\ecommerceproject\sql\marts\mart_traffic_source_quality.sql
✅ Ran: c:\dev\ecommerceproject\sql\marts\mart_fulfillment_deciles.sql
✅ Ran: c:\dev\ecommerceproject\sql\marts\mart_monthly_trends.sql
✅ Ran: c:\dev\ecommerceproject\sql\marts\mart_cohorts_new_vs_repeat.sql


## QC Checks

In [6]:
tables = [
  "thelook_analytics.mart_order_items_base",
  "thelook_analytics.mart_top_products",
  "thelook_analytics.mart_traffic_source_quality",
  "thelook_analytics.mart_fulfillment_deciles",
  "thelook_analytics.mart_monthly_overall",
  "thelook_analytics.mart_monthly_by_category",
  "thelook_analytics.mart_monthly_by_traffic_source",
  "thelook_analytics.mart_cohorts_new_vs_repeat",
]

for t in tables:
    print(t, "rows:", table_count(t))


thelook_analytics.mart_order_items_base rows: 95717
thelook_analytics.mart_top_products rows: 200
thelook_analytics.mart_traffic_source_quality rows: 5
thelook_analytics.mart_fulfillment_deciles rows: 10
thelook_analytics.mart_monthly_overall rows: 13
thelook_analytics.mart_monthly_by_category rows: 338
thelook_analytics.mart_monthly_by_traffic_source rows: 65
thelook_analytics.mart_cohorts_new_vs_repeat rows: 26
